In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import types
import pyspark

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("homework") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 10:46:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet -O ./data/yellow_tripdata_2024-10.parquet

In [3]:
df_spark_Yellow = spark.read.parquet('./data/yellow_tripdata_2024-10.parquet')

In [ ]:
df_repartitioned = df_spark.repartition(numPartitions=4)

In [ ]:
df_repartitioned.write.parquet('./data/partitions',mode='overwrite')

In [16]:
df_spark_Yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [17]:
df_spark_Yellow.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [5]:
df_spark_Yellow.createOrReplaceTempView ("yellow_taxi")

In [6]:
spark.sql('''
SELECT 
    COUNT(*)
FROM 
    yellow_taxi 
WHERE 
    DATE(tpep_pickup_datetime) = '2024-10-15'
''').show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [7]:
spark.sql('''
SELECT
    MAX(TIMESTAMPDIFF(SECOND,tpep_dropoff_datetime,tpep_pickup_datetime)/3600) AS longest_trip
FROM
    yellow_taxi
''').show()

+------------------+
|      longest_trip|
+------------------+
|23.784166666666668|
+------------------+



In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv -O ./data/taxi_zone_lookup.csv

In [12]:
df_spark_zones = spark.read\
        .option('header','True')\
        .csv('./data/taxi_zone_lookup.csv')

In [13]:
df_spark_zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [14]:
df_spark_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [21]:
df_spark_zones.createOrReplaceTempView('zones')

In [22]:
spark.sql('''
SELECT z.Zone, COUNT(y.PULocationID) AS trip_count
FROM yellow_taxi y
JOIN zones z ON y.PULocationID = z.LocationID
GROUP BY z.Zone
ORDER BY trip_count ASC
LIMIT 1
''').show()


+--------------------+----------+
|                Zone|trip_count|
+--------------------+----------+
|Governor's Island...|         1|
+--------------------+----------+

